# Hackathon Q1 - 2024

## Context

### Overall goal

Setting up a workflow to enable running RA2CE with multiple flood scenarios in an efficient way, on a larger geographical scale and post-processing the outcomes effectively into meaningful results


### User questions

- [UQ.1] Which roads are most likely to get hit by flooding from this hurricane given its projected flood maps?
- [UQ.2] Where are the flood impacts most disruptive in terms of accessibility? (detour length) = multi-link redundancy 
- [UQ.3] Which areas/locations are most likely to be unreachable because of this hurricane given its possible tracks? (OD analysis) (to be refined)
- [UQ.4] Optional if damages module works: What is the range of minimum and maximum damages that might occur because of this hurricane given its possible tracks?


### Hackathon requirements

- Being able to run RA2CE on a large scale with a complex network 
    - This should be coded generically so that we could do this ‘anywhere’
    - But computational time increases a lot with the scale and complexity of the network – how to handle?
    - How to determine the extent of the network with different flood maps?
    - Splitting up in smaller subsets? How would that change workflow and results?
- Running RA2CE in the cloud with multiple flood maps (100+) in different formats and storing the results
    - Being able to handle netcdf / data array / zar data formats?
    - Storing the different RA2CE runs and data efficiently
    - Skipping the second hazard overlay with the segmented graph as it increases computational time? 
    - Running multiple flood maps that represent a time series and then adding a time dimension in RA2CE results / analysis 
- Having a script that handles and post-processes multiple results 
    - Processing and storing results for all scenario runs and consolidate/merge them
    - Determining what the most interesting information is and how to communicate/visualize this
    - Visualization outputs such as statistics or ranges

### Workflow

The workflow to be run (with slight modifications) to solve all the [user questions](#user-questions) is as described in the following diagram:

| ![ra2ce_cloud.drawio.png](./user_question_1/ra2ce_cloud.drawio.png)| 
|:--:| 
| *Generic hackathon workflow* |

## User question 1

UQ.1: Which roads are most likely to get hit by flooding from this hurricane given its projected flood maps?

- __Input__: road extent (OSM), 100+ flood maps from SFINCS in Beira/Qualimane area and a RA2CE folder setup with .ini files.
- __Proposed workflow__: multiple flood maps – for every flood map a seperate RA2CE run - for every scenario seperate results – save the results in an efficient way – postprocess into meaningful results.
- __Expected output__: flood depths on edges for every scenario.
- __Expected post-processed results__: per edge an indication of ‘likelihood’ of flooding (e.g. in 90 / 100 scenario’s this edge gets hit (% hits)). 
- __Acceptance level__: Tackle user question 1 on Tuesday.
- Which questions for user questions 2,3,4 are also relevant for question 1?

### Input

- Collection of hazard files in `.tif` format.
- Boundary box (coordinates) of the network extent.
- ra2ce network configuratino file in `.ini` file.

### Pre-processing

#### Re-projecting

It might be possible that we require to pre-process the hazard files due to a different projection than WGS-84.

This can be done either locally or "in the cloud".

#### Creating the buckets

We create buckets online with each containing our network configuration, network extent and only one hazard file. This way we spread the computation of each hazard overlay for enhanced performance.

### Processing

#### Running the hazard overlay

In each bucket, we do a simple ra2ce run by modifying the `NetworkConfigData.hazard.hazard_map` property so that instead of having 'n' defined hazard files, contains only the name of the available hazard file for its executing "bucket".

### Post-processing

#### Unifying the outputs

Because we ran ra2ce with one container per hazard file, it means we have our output spread over different containers. We then unify all the available outputs and export its content into both a `.json` (geojson) and a `.feather` file.

### Visualizing

Last, we can now manually download the results ( `.json` and `.feather`) and visualize them locally.